<a href="https://colab.research.google.com/github/dwikagt/P2P_byProfit/blob/Master/99%2C98%25_36FS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [ ]:
! pip install -q kaggle

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy import stats 
import zipfile,os

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dwikaagt","key":"9557792337e4c4f134ed29bbfe845107"}'}

In [ ]:
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d muchazizmuslim/gafs-p25g50-p2p-lending-profit-score-dataset

100% 46.0M/46.2M [00:00<00:00, 152MB/s]
100% 46.2M/46.2M [00:00<00:00, 137MB/s]


In [ ]:
local_zip = '/content/gafs-p25g50-p2p-lending-profit-score-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
df = pd.read_csv ("/tmp/Best_Feature_P25_G50_Profit_Score_Dataset.csv")

In [ ]:
df['Profit_score'].value_counts()

1    2151725
0      11243
Name: Profit_score, dtype: int64

In [ ]:
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, cross_val_predict
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# Modeling

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.head()

,loan_amnt,term,int_rate,dti,revol_bal,last_pymnt_amnt,num_actv_rev_tl,pub_rec_bankruptcies,FICO,sub_grade_A2,...,verification_status_Source Verified,purpose_credit_card,purpose_educational,purpose_home_improvement,purpose_medical,purpose_moving,purpose_small_business,purpose_wedding,home_ownership_RENT,Profit_score
0,3600.0,36,13.99,5.91,2765.0,122.67,4.0,0.0,677.0,0,...,0,0,0,0,0,0,0,0,0,1
1,24700.0,36,11.99,16.06,21470.0,926.35,5.0,0.0,717.0,0,...,0,0,0,0,0,0,1,0,0,1
2,20000.0,60,10.78,10.78,7869.0,15813.30,3.0,0.0,697.0,0,...,0,0,0,1,0,0,0,0,0,1
3,35000.0,60,14.85,17.06,7802.0,829.90,5.0,0.0,787.0,0,...,1,0,0,0,0,0,0,0,0,1
4,10400.0,60,22.45,25.37,21929.0,10128.96,6.0,0.0,697.0,0,...,1,0,0,0,0,0,0,0,0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2162968 entries, 0 to 2162967
Data columns (total 37 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   loan_amnt                            float64
 1   term                                 int64  
 2   int_rate                             float64
 3   dti                                  float64
 4   revol_bal                            float64
 5   last_pymnt_amnt                      float64
 6   num_actv_rev_tl                      float64
 7   pub_rec_bankruptcies                 float64
 8   FICO                                 float64
 9   sub_grade_A2                         int64  
 10  sub_grade_A5                         int64  
 11  sub_grade_B5                         int64  
 12  sub_grade_C1                         int64  
 13  sub_grade_C4                         int64  
 14  sub_grade_D4                         int64  
 15  sub_grade_E3                    

In [ ]:
X = df.drop(['Profit_score'], axis=1)
y = df['Profit_score']

In [ ]:
clf1 = LGBMClassifier()
LGB = cross_val_score(clf1, X,y, cv=10, scoring = "accuracy")
y_pred = cross_val_predict(clf1, X,y, cv=10)
for i in range(10):
  print(f"Accuracy of LightGBM {i+1} : {LGB[i,] * 100:.2f}%")
print(f"Average Accuracy of LightGBM: {LGB.mean() * 100:.2f}%")
print(f"Best Accuracy of LightGBM: {LGB.max() * 100:.2f}%")
print(f"Confusion Matrix: \n {confusion_matrix(y, y_pred)}\n")

Accuracy of LightGBM 1 : 100.00%
Accuracy of LightGBM 2 : 100.00%
Accuracy of LightGBM 3 : 100.00%
Accuracy of LightGBM 4 : 99.80%
Accuracy of LightGBM 5 : 100.00%
Accuracy of LightGBM 6 : 100.00%
Accuracy of LightGBM 7 : 100.00%
Accuracy of LightGBM 8 : 100.00%
Accuracy of LightGBM 9 : 99.99%
Accuracy of LightGBM 10 : 100.00%
Average Accuracy of LightGBM: 99.98%
Best Accuracy of LightGBM: 100.00%
Confusion Matrix: 
 [[  11219      24]
 [    437 2151288]]



In [ ]:
clf2 = LGBMClassifier()
LGB = cross_val_score(clf2, X,y, cv=5, scoring = "accuracy")
y_pred = cross_val_predict(clf2, X,y, cv=5)
for i in range(5):
  print(f"Accuracy of LightGBM {i+1} : {LGB[i,] * 100:.2f}%")
print(f"Average Accuracy of LightGBM: {LGB.mean() * 100:.2f}%")
print(f"Best Accuracy of LightGBM: {LGB.max() * 100:.2f}%")
print(f"Confusion Matrix: \n {confusion_matrix(y, y_pred)}\n")

Accuracy of LightGBM 1 : 100.00%
Accuracy of LightGBM 2 : 99.90%
Accuracy of LightGBM 3 : 99.99%
Accuracy of LightGBM 4 : 100.00%
Accuracy of LightGBM 5 : 99.99%
Average Accuracy of LightGBM: 99.98%
Best Accuracy of LightGBM: 100.00%
Confusion Matrix: 
 [[  11219      24]
 [    495 2151230]]

